In [1]:
import test_similarity

In [2]:
import pandas as pd

import numpy as np
from numpy import dot
from numpy.linalg import norm
cos_sim = lambda a,b: dot(a, b)/(norm(a)*norm(b))

import matplotlib.pyplot as plt

In [3]:
def predict(source_vecs, target_vecs, top_n=3):
    top_n_predictions_all = []
    for sv in source_vecs:
        sims = [cos_sim(sv, tv) for tv in target_vecs]
        top_n_predictions_single = [idx2ps[idx] for idx in np.argsort(sims)[::-1][:top_n]]
        top_n_predictions_all.append(top_n_predictions_single)
    return top_n_predictions_all   

def evaluate(preds, gts):
    n = 0
    for i, pred in enumerate(preds):
        if gts[i] in pred:
            n+=1           
    return float("%.3f" %(n/len(preds)))

def repeat_data(ps_unique, mg, ps, n=2):
    _ps_unique_repeat = [(p+' ')*n for p in ps_unique]
    _mg_repeat = [(m+' ')*n for m in mg]
    _ps_repeat = [(p+' ')*n for p in ps]
    return _ps_unique_repeat, _mg_repeat, _ps_repeat

def repeat_data2(ps_unique, mg, ps, n=2):
    _ps_unique_repeat = [(p+' in construction ')*n for p in ps_unique]
    _mg_repeat = [(m+' in construction ')*n for m in mg]
    _ps_repeat = [(p+' in construction ')*n for p in ps]
    return _ps_unique_repeat, _mg_repeat, _ps_repeat

In [4]:
data = pd.read_excel('../data/Case1_Dataset.xlsx')
MG = list(data['Material Group Description']);PS = list(data['Procurement Structure Description'])

In [5]:
ps_unique = list(set(PS))

#ps_unique, MG, PS = repeat_data(ps_unique, MG, PS, n=30)
ps_unique, MG, PS = repeat_data2(ps_unique, MG, PS, n=8)

idx2ps = {k:v for k,v in enumerate(ps_unique)}
ps2idx = {v:k for k,v in enumerate(ps_unique)}

idx2mg = {k:v for k,v in enumerate(MG)}
mg2idx = {v:k for k,v in enumerate(MG)}

In [6]:
from importlib import reload
reload(test_similarity)

model = test_similarity.Model(vocab_file='../model_kb/vocab.txt', 
              bert_config_file='../model_kb/bert_config.json', 
              init_checkpoint='../model_kb/bert_model.ckpt', 
              model_dir='../model_kb/',
              layer=[-1, -2], 
              pooling_strategy = 'MEAN',
              max_seq_length=26)

In [7]:
print(model.similarity("river bank", "mechant bank"))
%time model.similarity("windows operating system", "windows platform architecture")

0.6063
Wall time: 447 ms


0.7848

In [8]:
all_mg_vecs = model.encode(MG);all_ps_vecs = model.encode(ps_unique)

In [9]:
accs = []

predictions = predict(all_mg_vecs, all_ps_vecs, top_n=1)
acc1 = evaluate(predictions, PS)

accs.append(acc1)
print("Top1 accuracy: ", acc1)

Top1 accuracy:  0.535


In [10]:
top_mappings = predict(all_mg_vecs, all_ps_vecs, top_n=3)
acc3 = evaluate(top_mappings, PS)
accs.append(acc3)
print("Top3 accuracy: ", acc3)

Top3 accuracy:  0.701


In [11]:
top_mappings = predict(all_mg_vecs, all_ps_vecs, top_n=5)
acc5 = evaluate(top_mappings, PS)
accs.append(acc5)
print("Top5 accuracy: ", acc5)

Top5 accuracy:  0.803
